In [ ]:
from huggingface_hub import snapshot_download
from moondream_model import VisionEncoder, TextModel
import torch 
from PIL import Image
import re

model_path = snapshot_download("vikhyatk/moondream1")

DEVICE = "cuda"
DTYPE = torch.float16

vision_encoder = VisionEncoder(model_path).to(DEVICE, dtype=DTYPE)
text_model = TextModel(model_path).to(DEVICE, dtype=DTYPE)

print(f"Using:{DEVICE}")
print(f"Type: {DTYPE}")

In [ ]:
def run_inference(image, prompt):
    with torch.inference_mode(), torch.cuda.amp.autocast():
        image_embeds =vision_encoder(image).to(DEVICE, dtype=DTYPE)
        result = text_model.answer_question(image_embeds, prompt)
    
    if isinstance(result, tuple):
        result_text = result[0]
    else:
        result_text = result

    # Convert the result to string if it's not already
    if not isinstance(result_text, str):
        if torch.is_tensor(result_text):
            result_text = result_text.cpu().numpy().tolist()
            result_text = ' '.join(map(str, result_text))
        else:
            result_text = str(result_text)

    # Apply regex to clean up the result string
    cleaned_result = re.sub("<$", "", re.sub("END$", "", result_text))
    return cleaned_result

In [ ]:
img = Image.open("img/output_000017.jpg")
prompt = "Describe this image."
result = run_inference(img, prompt)
result